# Final Project
#### This notebook simulates the behaviour of a carwash
#### given a set of interarrival time and service time
#### we will find the statistic metrics, the distribution and analyze the results.

In [2]:
from numpy import mean
from scipy import stats
import numpy as np
from random import expovariate
import matplotlib.pyplot as plt

# Load the data:

In [3]:
dataset = np.loadtxt("data.txt", delimiter=',')
arrives = dataset[:,0]
services = dataset[:,1]
it = np.array([i for i in range(len(arrives))])

# Find p value for arrives and services:

In [11]:
Slope, Intercept, RValue, PValue, StdErr = stats.linregress(dataset)

# Auto correlation

In [12]:
if(PValue>0.05):
    print("The resulting p value for arrives is: {} and its greater than 0.05.\n".format(PValue))
    r = (np.correlate(it-it.mean(),it-it.mean(), mode='full'))/(it.var()*len(it))
    arrivesAutoCor = r[r.size//2:]
    print("Autocorrelation for arrives is: ", arrivesAutoCor)
else:
    print("P value for arrives is {}, the condition is not met.".format(PValue))

The resulting p value for arrives is: 0.30076797128732774 and its greater than 0.05.

Autocorrelation for arrives is:  [ 1.          0.9         0.80044494  0.70177976  0.60444939  0.50889878
  0.41557286  0.32491657  0.23737486  0.15339266  0.07341491 -0.00211346
 -0.0727475  -0.13804227 -0.19755284 -0.25083426 -0.2974416  -0.33692992
 -0.36885428 -0.39276974 -0.40823137 -0.41479422 -0.41201335 -0.39944383
 -0.37664071 -0.34315907 -0.29855395 -0.24238042 -0.17419355 -0.09354839]


In [13]:
#Set the distributions list
distributions = [
    "gamma",
    "expon",
    "lognorm",
    "weibull_min",  
    "weibull_max",
    "norm"
]

# Find distribution

## for arrives

In [16]:
arrivesDistributionResults = []
for distName in distributions:
    dist = getattr(stats, distName)
    fittedData = dist.fit(arrives)
    D, p = stats.kstest(arrives, distName, args=fittedData);
    arrivesDistributionResults.append((distName,p,fittedData))
auxP = 0
arrivesAuxDist = ""
print("P Values for each distribution and fit parameter: \n")
for distribution in arrivesDistributionResults:
    print("{}: {}".format(distribution[0], distribution[1]))
    if(auxP<distribution[1]):
        arrivesAuxDist = distribution[0]
        auxP = distribution[1]
        arrivesAuxParams = distribution[2]
print("\nThe correct distribution for the dataset is {0} as it has the highest p value ({1}).".format(arrivesAuxDist,auxP))

P Values for each distribution and fit parameter: 

gamma: 0.017726561468500712
expon: 0.8575377339716195
lognorm: 0.8205905276555825
weibull_min: 0.001076672930113265
weibull_max: 2.220446049250313e-16
norm: 0.39714998744337104

The correct distribution for the dataset is expon as it has the highest p value (0.8575377339716195).


## for services

In [17]:
servicesDistributionResults = []
for distName in distributions:
    dist = getattr(stats, distName)
    fittedData = dist.fit(services)
    D, p = stats.kstest(services, distName, args=fittedData);
    servicesDistributionResults.append((distName,p,fittedData))
auxP = 0
servicesAuxDist = ""
print("P Values for each distribution:\n")
for distribution in servicesDistributionResults:
    print("{}: {}".format(distribution[0], distribution[1]))
    if(auxP<distribution[1]):
        servicesAuxDist = distribution[0]
        auxP = distribution[1]
        servicesAuxParams = distribution[2]
print("\nThe correct distribution for the dataset is {0} as it has the highest p value ({1}).".format(servicesAuxDist,auxP))

P Values for each distribution:

gamma: 0.992276174713438
expon: 0.08448985297392575
lognorm: 1.4741541320972829e-12
weibull_min: 1.5543122344752192e-15
weibull_max: 0.0
norm: 0.9945787734080286

The correct distribution for the dataset is norm as it has the highest p value (0.9945787734080286).


In [18]:
arrivalsDist = getattr(stats, arrivesAuxDist)
servicesDist = getattr(stats, servicesAuxDist)

In [55]:
#This block has the function that is intended to calculate the arrival and service time in a random way
#based on the amount of minutes of work in a day.

##It returns two lists: arrival times and services times.

def generateArrivalAndServiceTimes():
    workingDay = 9*60 #9 hours expressed in minutes.
    arrivals = [0] #This will contain the arrival times for n works in a day.
    services = [0] #This will contain the service times for n works in a day.
    totalAmountOfHours = 0
    while totalAmountOfHours < workingDay:
        nArrivalTime = arrivalsDist.rvs(loc=arrivesAuxParams[-2], scale=arrivesAuxParams[-1])
        nServiceTime = servicesDist.rvs(loc=servicesAuxParams[-2], scale=servicesAuxParams[-1])
        totalAmountOfHours += nServiceTime
        arrivals.append(nArrivalTime)
        services.append(nServiceTime)
    return arrivals, services

In [59]:
#This blocks contains the delay time function

##It returns two lists: delay time, and departure time.

def delayTime(a, s):
    c = [0.0]*len(a)
    i = 0
    d = [0.0]*len(a)
    while i < len(a)-1:
        i+=1
        if a[i] < c[i-1]:
            d[i] = c[i-1] - a[i]
        else:
            d[i] = 0
        c[i] = a[i] + d[i] + s[i]
    return d, c

In [60]:
#This block contains the job-averaged statistics

def averageInterarrival(a):
    r = mean(a)
    aR = 1/r
    return r, aR

def averageService(s):
    s = mean(s)
    sR = 1/s
    return s, sR

def averageDelay(delay):
    d = mean(delay)
    return d

def averageWait(d, s):
    w = mean(d) + mean(s)
    return w

In [61]:
#This block contains the time-averaged statistics using little's theorem.

#Ew //Sumatory of w divided by thao.
#It recieves three parameters: delay times (list), service times (list), departure times (list).
def little_jobsInServiceNode(d, s, c):
    w = 0.0
    for i, j in zip(d, s):
        w+=(i+j)
    return w/c[-1]

#Ed //Sumatory of d divided by thao
#It recieves two parameters: delay times (list), departure times (list).
def little_jobsInQueue(d, c):
    sumD = 0.0
    for i in d:
        sumD+=i
    return sumD/c[-1]

#Es //Sumatory of s divided by thao
#It recieves two parameters: service times (list), departure times (list).
def little_jobsInService(s, c):
    sumS = 0.0
    for i in s:
        sumS+=i
    return sumS/c[-1]

In [62]:
#This blocks contains the interval estimation function.
#Recieves a list and a number as parameters.
def intervalEstimation(data, confidence):
    xMean = mean(data)
    s2 = 0
    for i in data:
        s2 += (i - xMean)**2  
    s2 = s2/len(data)
    s = s2**(1/2) #Standard deviation
    alpha = (100 - confidence)/100
    t = 1-alpha/2
    
    start = xMean - (t * s)/(len(data)-1)**(1/2)
    end = xMean + (t * s)/(len(data)-1)**(1/2)
    
    interval = {'start': start, 'end': end}
    return interval

In [71]:
amountOfJobs = []

avgInterarrivals = []; arrivalRate = []; avgServices = []; serviceRate = []; avgDelays = []; avgWaits = []

jbsInServiceNode = []; jbsInQueue = []; jbsInService = []
for i in range(30):
    arrivals, services = generateArrivalAndServiceTimes()
    amountOfJobs.append(len(arrivals))
    delays, c = delayTime(arrivals, services)
    aI, aR = averageInterarrival(arrivals)
    avgInterarrivals.append(aI); arrivalRate.append(aR)
    
    aS, sR = averageService(services)
    avgServices.append(aS); serviceRate.append(sR)
    
    avgDelays.append(averageDelay(delays)); avgWaits.append(averageWait(delays, services))
    
    jbsInServiceNode.append(little_jobsInServiceNode(delays, services, c))
    jbsInQueue.append(little_jobsInQueue(delays, c))
    jbsInService.append(little_jobsInService(services, c))    

print("###########  Results ############\n")
print("#### Amount of jobs intervals: {}\n".format(intervalEstimation(amountOfJobs, 95)))

print("#### Job-average statistics intervals: \n")
print("Average interarrival time intervals: {} | Arrivals rate intervals: {}\n".format(intervalEstimation(avgInterarrivals, 95), intervalEstimation(arrivalRate, 95)))
print("Average service time intervals: {} | Service rate intervals: {}\n".format(intervalEstimation(avgServices, 95), intervalEstimation(serviceRate, 95)))
print("Average delay time intervals: {}\n".format(intervalEstimation(avgDelays, 95)))
print("Average wait time intervals: {}\n".format(intervalEstimation(avgWaits, 95)))

print("#################################\n")
print("#### Time-average statistics intervals: \n")
print("Intervals for time in service node: {}\n".format(intervalEstimation(jbsInServiceNode, 95)))
print("Intervals for time in queue: {}\n".format(intervalEstimation(jbsInQueue, 95)))
print("Intervals for time in service: {}\n".format(intervalEstimation(jbsInService, 95)))

###########  Results ############

#### Amount of jobs intervals: {'start': 9.042161693386925, 'end': 9.357838306613074}

#### Job-average statistics intervals: 

Average interarrival time intervals: {'start': 15.695092476861186, 'end': 17.49717825687081} | Arrivals rate intervals: {'start': 0.06226705676215353, 'end': 0.06934217493290497}

Average service time intervals: {'start': 61.78365301476109, 'end': 63.74213511605736} | Service rate intervals: {'start': 0.015795300463714645, 'end': 0.01631690819146481}

Average delay time intervals: {'start': 215.13797733768533, 'end': 222.6174911222195}

Average wait time intervals: {'start': 278.1903689513915, 'end': 285.09088763933164}

#################################

#### Time-average statistics intervals: 

Intervals for time in service node: {'start': 4.314535695572376, 'end': 4.498251292013531}

Intervals for time in queue: {'start': 3.3400526363859644, 'end': 3.524433184012806}

Intervals for time in service: {'start': 0.970810078620